In [1]:
import torch
import peewee
import pandas as pd
import torch.optim as optim
import torch.nn as nn

In [2]:
import os, sys

root_dir = os.path.dirname("../")
print(root_dir)

if root_dir not in sys.path:
    sys.path.append(root_dir)
    print("Path added")
else:
    print("Path exists")

..
Path added


In [3]:
from src.model.data import create_dataloaders
from src.model.main import LSTM

In [4]:
device = "cuda" if torch.cuda.is_available() else "cpu"
device

'cuda'

#### Prepare Data

In [5]:
db = peewee.SqliteDatabase("../data/dataset.sqlite3")
conn = db.connection()
tables = db.get_tables()

In [6]:
stock = "AAPL"
stock_df = pd.read_sql(f"SELECT * FROM {tables[0]} WHERE symbol = '{stock}'", conn)
stock_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 338142 entries, 0 to 338141
Data columns (total 7 columns):
 #   Column     Non-Null Count   Dtype  
---  ------     --------------   -----  
 0   symbol     338142 non-null  object 
 1   timestamp  338142 non-null  object 
 2   open       338142 non-null  float64
 3   high       338142 non-null  float64
 4   low        338142 non-null  float64
 5   close      338142 non-null  float64
 6   volume     338142 non-null  float64
dtypes: float64(5), object(2)
memory usage: 18.1+ MB


#### Hyperparameters

In [7]:
input_size = 5  # OHLCV
hidden_size = 50
num_layers = 2
output_size = 5  # Predicting OHLCV

seq_length = 10
batch_size = 32
test_size = 0.2
num_epochs = 10
learning_rate = 0.001

In [8]:
train_dataloader, test_dataloader = create_dataloaders(stock_df, seq_length, batch_size, test_size)

Cols ['open', 'high', 'low', 'close', 'volume']


In [9]:
model = LSTM(input_size, hidden_size, num_layers, output_size, device)

In [10]:
device = model.device
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

for epoch in range(num_epochs):
    model.train()
    total_loss = 0

    for batch, (inputs, targets) in enumerate(train_dataloader):
        inputs, targets = inputs.float().to(device), targets.float().to(device)

        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    average_loss = total_loss / len(train_dataloader)
    print(f"Epoch [{epoch + 1}/{num_epochs}], Loss: {average_loss:.4f}")

print("Training finished.")

tensor([[[2.0033e-01, 1.9989e-01, 2.0199e-01, 2.0075e-01, 2.1918e-06],
         [2.0026e-01, 1.9989e-01, 2.0173e-01, 2.0075e-01, 2.0905e-05],
         [2.0031e-01, 1.9987e-01, 2.0197e-01, 2.0073e-01, 1.5370e-06],
         ...,
         [2.0009e-01, 1.9977e-01, 2.0175e-01, 2.0061e-01, 8.7672e-06],
         [2.0019e-01, 1.9980e-01, 2.0178e-01, 2.0054e-01, 6.0929e-06],
         [2.0024e-01, 1.9980e-01, 2.0190e-01, 2.0066e-01, 3.0051e-06]],

        [[1.1992e-01, 1.1966e-01, 1.2058e-01, 1.1968e-01, 1.0792e-01],
         [1.1920e-01, 1.1922e-01, 1.2003e-01, 1.1893e-01, 1.9579e-04],
         [1.1849e-01, 1.1847e-01, 1.2004e-01, 1.1910e-01, 9.6542e-05],
         ...,
         [1.1699e-01, 1.1741e-01, 1.1826e-01, 1.1813e-01, 5.0927e-04],
         [1.1739e-01, 1.1922e-01, 1.1881e-01, 1.1764e-01, 1.4236e-04],
         [1.1710e-01, 1.1704e-01, 1.1850e-01, 1.1757e-01, 1.0614e-04]],

        [[3.0450e-01, 3.0449e-01, 3.0579e-01, 3.0476e-01, 3.9283e-03],
         [3.0446e-01, 3.0431e-01, 3.0598e-01,

RuntimeError: Input and parameter tensors are not at the same device, found input tensor at cuda:0 and parameter tensor at cpu